In [1]:
import json
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import mplhep as mh

from hist.hist import Hist
from hist.axis import Regular

from pathlib import Path

from typing import Optional, Union

In [2]:
from functools import cache
from functools import cached_property

@cache
def get_segment(ring: int, station: int, sector: int, subsector: int) -> int:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L361-L368
    """
    nsub = 3 if ring == 1 and station > 1 else 6
    return subsector + nsub * (sector - 1)


@cache
def get_roll_name(region: int, ring: int, station: int, sector: int, layer: int,
             subsector: int, roll: int
) -> str:
    """
    https://github.com/cms-sw/cmssw/blob/CMSSW_13_3_0_pre3/Geometry/RPCGeometry/src/RPCGeomServ.cc#L11-L87
    """
    if region == 0:
        name = f'W{ring:+d}_RB{station}'

        if station <= 2:
            name += 'in' if layer == 1 else 'out'
        else:
            if sector == 4 and station == 4:
                name += ['--', '-', '+', '++'][subsector - 1]
            elif (station == 3) or (station == 4 and sector not in (4, 9, 11)):
                name += '-' if subsector == 1 else '+'
        name += f'_S{sector:0>2d}_'
        name += ['Backward', 'Middle', 'Forward'][roll - 1]
    else:
        segment = get_segment(ring, station, sector, subsector)
        name = f'RE{station * region:+d}_R{ring}_CH{segment:0>2d}_'
        name += ['A', 'B', 'C', 'D', 'E'][roll - 1]
    return name

In [11]:
def root2csv(input_path: Path,
                     output_path: Path,
)->dict:
    data = uproot.open(f"{str(input_path)}:tree").arrays(library='np')

    data['roll_name'] = np.array([get_roll_name(
                                                data['region'][idx], data['ring'][idx], data['station'][idx],
                                                data['sector'][idx], data['layer'][idx], data['subsector'][idx], data['roll'][idx]) 
                                                for idx in range(len(data['region']))])
    
    data2segment = np.vectorize(get_segment)
    data['segment'] = data2segment(data['ring'], data['station'],
                                               data['sector'], data['subsector'])

    data =  pd.DataFrame.from_dict(data=data, orient='columns')
    data.to_csv(output_path, index=False)


def root2df(
    input_path: Path,
)->dict:
    data = uproot.open(f"{str(input_path)}:tree").arrays(library='np')

    data['roll_name'] = np.array([get_roll_name(
                                                data['region'][idx], data['ring'][idx], data['station'][idx],
                                                data['sector'][idx], data['layer'][idx], data['subsector'][idx], data['roll'][idx]) 
                                                for idx in range(len(data['region']))])
    
    data2segment = np.vectorize(get_segment)
    data['segment'] = data2segment(data['ring'], data['station'],
                                               data['sector'], data['subsector'])

    data =  pd.DataFrame.from_dict(data=data, orient='columns')
    return data

def load_data(
    input_path: Path,
)->dict:
    data = uproot.open(f"{str(input_path)}:tree").arrays(library='df')

    data['roll_name'] = np.array([get_roll_name(
                                                data['region'][idx], data['ring'][idx], data['station'][idx],
                                                data['sector'][idx], data['layer'][idx], data['subsector'][idx], data['roll'][idx]) 
                                                for idx in range(len(data['region']))])
    
    data2segment = np.vectorize(get_segment)
    data['segment'] = data2segment(data['ring'], data['station'],
                                               data['sector'], data['subsector'])

    data =  pd.DataFrame.from_dict(data=data, orient='columns')
    return data



In [32]:
all_columns = [
    'is_fiducial', 'is_matched', 
    'region', 'ring', 'station', 'sector', 'layer', 'subsector', 'roll', 
    'run', 'cls', 'bx', 
    'tag_pt', 'tag_eta', 'tag_phi', 
    'probe_pt', 'probe_eta', 'probe_phi', 'probe_time', 'probe_dxdz', 'probe_dydz', 
    'dimuon_pt', 'dimuon_mass', 
    'residual_x', 'residual_y', 'pull_x', 'pull_y', 'pull_x_v2', 'pull_y_v2', 
]

columns = [
    'is_fiducial', 'is_matched', 
    'region', 'ring', 'station', 'sector', 'layer', 'subsector', 'roll', 
    'cls', 'bx', 'run'
] 

In [34]:
working_dir = Path('/users/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/TnP_Plotting')
input_path = working_dir / 'data' / 'Muon__Run2022C.root'

data = uproot.open(f"{str(input_path)}:tree").arrays(columns, library='pd')

In [44]:
print(data.columns)
print(data.size)

Index(['is_fiducial', 'is_matched', 'region', 'ring', 'station', 'sector',
       'layer', 'subsector', 'roll', 'cls', 'bx', 'run'],
      dtype='object')
140548032
